Ce notebook contient les différentes fonctions utilisées pour la mise en œuvre de l'algorithme LBG, également connu sous le nom d'algorithme de Lloyd généralisé. 

On commence par importer les différentes librairies nécessaires :

In [ ]:
import math
from functools import reduce
from collections import defaultdict

data_size = 0
dim = 0

Les fonctions implémentées sont les suivantes :

genCodebook : génère un codebook (ensemble de vecteurs de taille N) à partir d'un ensemble de vecteurs d'apprentissage donné.  On itère jusqu'à ce que a convergence soit atteinte.

In [ ]:
def genCodebook(data, N, epsilon):
    global data_size
    global dim
    
    data_size = len(data)
    assert data_size > 0
    
    dim = len(data[0])
    assert dim > 0
    
    codebook = []
    codebook_weights_abs = [data_size]
    codebook_weights_rel = [1.0]
    
    # On calcule le vecteur initial du codebook : on prend la moyenne de tous les vecteurs de notre image
    c0 = moy_vec_de_vecs(data, dim, data_size)
    codebook.append(c0)

moy_vec_de_vecs : calcule la moyenne de tous les vecteurs d'un ensemble de vecteurs donné.

In [ ]:
def avg_vec_of_vecs(vecs, dim=None, size=None):
    """
    Calculcate average vector (center vector) for input vectors <vecs>.
    :param vecs: input vectors
    :param dim: dimension of <vecs> if it was already calculated
    :param size: size of <vecs> if it was already calculated
    :return average vector (center vector) for input vectors <vecs>
    """
    size = size or len(vecs)
    dim = dim or len(vecs[0])
    avg_vec = [0.0] * dim
    for vec in vecs:
        for i, x in enumerate(vec):
            avg_vec[i] += x / size

    return avg_vec